In [3]:
# %%
# Imports
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import json
import shutil
import torch
from sklearn import metrics as skmetrics
from argparse import ArgumentParser

# Imports from this project
sys.path.insert(0, "..")
from utils import paths
from utils.input_output import load_feature_keys, load_feature_properties, load_preprocessed_data
from utils.histograms import find_good_binning, get_hist, calc_pull
from utils.merge_pdfs import merge_pdfs
from model_B_classifier import DeepSetModel

# Evaluation Results from different models

In [6]:
with open("../features_B_classifier.json", "r") as file:
    feature_lists = json.load(file)
    
trained_model_names = [n.replace("features_","") for n in feature_lists.keys() if (paths.models_dir/n.replace("features_","")/paths.B_classifier_eval_file.name).is_file()]
trained_model_names

['B_classifier_all',
 'B_classifier_without_Tr_T_x',
 'B_classifier_without_Tr_ProbSS',
 'B_classifier_without_Tr_T_ACHI2DOCA',
 'B_classifier_without_Tr_T_ADOCA',
 'B_classifier_without_Tr_T_Charge']

In [15]:
df_results = pd.DataFrame()

for model_name in trained_model_names:
    paths.update_B_classifier_name(model_name)
    with open(paths.B_classifier_dir/"eval_results.json", "r") as file:
        eval_results = json.load(file)
        
    for metric in eval_results.keys():
        if metric != "confusion_matrix_test":
            df_results.loc[model_name,metric] = eval_results[metric]
            
    if "confusion_matrix_test" in eval_results.keys():
        df_results.loc[model_name,"efficiency_Bd_test"] = eval_results["confusion_matrix_test"][0][0]
        df_results.loc[model_name,"efficiency_Bs_test"] = eval_results["confusion_matrix_test"][1][1]
        
with pd.option_context("display.float_format", '${:,.3f}'.format):
    print(df_results.sort_values(by="accuracy_test", ascending=False))

                                     roc_auc_test  roc_auc_train  \
B_classifier_without_Tr_ProbSS             $0.931         $0.953   
B_classifier_all                           $0.934         $0.954   
B_classifier_without_Tr_T_x                $0.926         $0.946   
B_classifier_without_Tr_T_ADOCA            $0.913         $0.934   
B_classifier_without_Tr_T_ACHI2DOCA        $0.904         $0.930   
B_classifier_without_Tr_T_Charge           $0.891         $0.917   

                                     accuracy_test  accuracy_train  \
B_classifier_without_Tr_ProbSS              $0.857          $0.882   
B_classifier_all                            $0.854          $0.881   
B_classifier_without_Tr_T_x                 $0.846          $0.870   
B_classifier_without_Tr_T_ADOCA             $0.831          $0.857   
B_classifier_without_Tr_T_ACHI2DOCA         $0.829          $0.856   
B_classifier_without_Tr_T_Charge            $0.796          $0.820   

                                

# Generare Feature Lists

In [16]:
with open("../features_B_classifier.json", "r") as file:
    feature_lists = json.load(file)
    
assert "features_B_classifier_baseline" in feature_lists.keys()
assert "features_B_classifier_all" in feature_lists.keys()

baseline_features = feature_lists["features_B_classifier_baseline"]
all_features = feature_lists["features_B_classifier_all"]

In [17]:
# add lists with one feature added, each
for feature in set(all_features) - set(baseline_features):
    feature_lists[f"features_B_classifier_baseline_with_{feature}"] = baseline_features + [feature]

In [18]:
with open("../features_B_classifier.json", "w") as file:
    json.dump(feature_lists, file, indent=2)